In [16]:
import scvi
import numpy as np
import scanpy as sc
import random
import torch

In [2]:
def set_seed(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed) 
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False  

set_seed(42)

In [3]:
rna = sc.read("./data/neurips-multiome/rna_hvg.h5ad")
atac = sc.read("./data/neurips-multiome/atac_hvf.h5ad")

rna.var['modality'] = 'Gene Expression'
atac.var['modality'] = 'Peaks'

rna.X = rna.layers['counts']
atac.X = atac.layers['binary']

adata = rna.T.concatenate(atac.T).T

adata_mvi = scvi.data.organize_multiome_anndatas(adata)

adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()

scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key='modality', categorical_covariate_keys=['batch'])

mvi = scvi.model.MULTIVI(adata_mvi, n_genes=(adata_mvi.var['modality']=='Gene Expression').sum(), n_regions=(adata_mvi.var['modality']=='Peaks').sum())

mvi.train(batch_size=512)

adata_mvi.obsm["latent"] = mvi.get_latent_representation()

adata_mvi.write('./results/neurips-multiome-multivi.h5ad')

In [3]:
rna = sc.read("./data/neurips-cite/rna_hvg.h5ad")
protein = sc.read('./data/neurips-cite/protein.h5ad')

adata = rna
adata.X = rna.layers['counts'].toarray().copy()
adata.obsm['proteins'] = protein.layers['counts'].toarray().copy()

scvi.model.TOTALVI.setup_anndata(adata, batch_key='batch', protein_expression_obsm_key='proteins')

vae = scvi.model.TOTALVI(adata, latent_distribution="normal")

vae.train()

adata.obsm["latent"] = vae.get_latent_representation()

adata.write('./results/neurips-cite-totalvi.h5ad')